In [43]:
## Load the dataset and return the first few rows
import glob
import pandas as pd
import numpy as np
pd.options.display.max_columns = None
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,8)

Set up a regex to find the year from the filename

In [2]:
import re
year_match = re.compile('(\d{4})')

Couple of functions to transform the scoreline into useful data

In [3]:
def goals(result: str):
    '''Given a scoreline such as 6-1 returns home and away goals in a dict {home: 6, away: 1}'''
    nums = result.split('-')
    try:
        return {'home': int(nums[0]), 'away': int(nums[1])}
    except Exception:
        return {'home': -1, 'away': -1}

In [4]:
def outcome(result):
    '''Given a string of the result e.g. 6-1 categorise as either a no-score draw (1), score draw (2), home win (3) or away win (4)'''
    goals_result = goals(result)
    if goals_result['home'] == goals_result['away']:
        if goals_result['home'] == 0:
            return 1
        else:
            return 2
    elif goals_result['home'] > goals_result['away']:
        return 3
    else:
        return 4

Generate a list of available leagues from the folder names

In [5]:
league_list = glob.glob('/Users/dev/aicore/Football-predictor/Football-Dataset/*')
league_list = pd.DataFrame(league_list)
league_list.columns = ['folder']
league_list['name'] = league_list['folder'].apply(lambda x: x.split('/')[-1])

Get all files in a particular folder and list their names

In [6]:
def get_files(folder):
    league_file_list = glob.glob(folder + '/*')
    league_file_list.sort()
    return league_file_list
league_list['files'] = league_list['folder'].apply(get_files)

Now run through the list of leagues (folders) and the files in them (in the files column), read in the data, and combine them together into dfs.

Results:

- `overall_team_counts` is a df showing the number of teams involved per league per season (unlikely to be consistent - and indeed not - EPL is a counterexample)
- `df_full` is the whole data of results across all leagues and seasons

In [7]:
overall_team_counts = pd.DataFrame()
df_league_lists = []
for index, league in league_list.iterrows():
    league_file_list = league['files']
    team_counts = {}
    df_list = []
    for file in league_file_list:
        df = pd.read_csv(file)
        df.columns = df.columns.str.lower()
        year = year_match.findall(file)[0]
        if len(df.index) == 0:
            # print('No data for {}'.format(year))
            pass
        else:
            df_original = df.copy()
            df_list.append(df)
            team_counts[year] = len(df['home_team'].unique())
            # print('In {} there were {} teams'.format(year, len(df['Home_Team'].unique())))
    overall_team_counts = pd.concat([overall_team_counts, pd.Series(team_counts, name=league['name'])], axis=1)
    df_league_lists.append(pd.concat(df_list, ignore_index=True))
df_full = pd.concat(df_league_lists, ignore_index=True)
overall_team_counts.sort_index()
overall_team_counts = overall_team_counts.fillna(0).astype(int)

Use the functions defined earlier - `goals` and `outcome` - to transform the scoreline (as a string) into numerical data

In [8]:
df_full['goals'] = df_full['result'].apply(goals)
# in case we're repeating this operation
try:
    df_full = df_full.drop(['home','away'],axis=1) 
except Exception:
    pass
df_full = pd.concat([df_full, pd.json_normalize(df_full['goals'])], axis=1).drop('goals', axis=1)
# drop errors
df_full = df_full.drop(df_full[df_full.home == -1].index)
df_full['total_goals'] = df_full['home'] + df_full['away']
df_full['outcome'] = df_full['result'].apply(outcome)

In [9]:
df = df_full.copy()

## Additional datasets - join them on

## first: match info

In [10]:
match_df = pd.read_csv('additional-data/Match_Info.csv')
match_df.columns = match_df.columns.str.lower()
# match_df.info()
match_df['referee'] = match_df['referee'].str.strip().str.replace('Referee: ','')
match_df['date_dt'] = pd.to_datetime(match_df['date_new'])
match_df['link_parts'] = match_df['link'].str.split('/')
match_df['home_team'] = match_df.apply(lambda row: row['link_parts'][-3], axis=1)
match_df['away_team'] = match_df.apply(lambda row: row['link_parts'][-2], axis=1)
match_df['season'] = match_df.apply(lambda row: row['link_parts'][-1], axis=1)
match_df = match_df.drop(['date_new'], axis=1)

In [11]:
import difflib
df_teams = df['home_team'].unique()
match_teams = match_df['home_team'].unique()

Let's use Levenshtein. see https://www.datacamp.com/community/tutorials/fuzzy-string-python

In [12]:
from fuzzywuzzy import process
# needs a tweak for Manchester to give a helping hand on those two teams
matched_teams = [(d,(process.extractOne(d.replace('Man.','Manchester'), match_teams))[0]) for d in df_teams]

In [13]:
def team_name_for_link(t):
    matches = [x[1] for x in matched_teams if x[0] == t]
    return (matches if len(matches)>0 else [''])[0]
df['link'] = df.apply(lambda row: '/'.join(['','match',team_name_for_link(row['home_team']),team_name_for_link(row['away_team']),str(row['season'])]).strip().lower().replace(' ','-'), axis=1)

In [14]:
# duplicates?
print(df['link'].duplicated().value_counts())
print(df[['home_team','away_team','season','league']].duplicated().value_counts())
df.drop_duplicates(subset=['home_team','away_team','season','league'], inplace=True)

False    130890
True      15602
Name: link, dtype: int64
False    132103
True      14389
dtype: int64


In [15]:
df2 = pd.merge(match_df, df, how="inner", on='link')

## add team info

In [16]:
team_df = pd.read_csv('additional-data/Team_Info.csv')
team_df.columns = team_df.columns.str.lower()
team_df['capacity'] = pd.to_numeric(team_df['capacity'].str.replace(',',''))
team_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 544 entries, 0 to 543
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   team      544 non-null    object
 1   city      544 non-null    object
 2   country   544 non-null    object
 3   stadium   447 non-null    object
 4   capacity  544 non-null    int64 
 5   pitch     447 non-null    object
dtypes: int64(1), object(5)
memory usage: 25.6+ KB


In [17]:
teams_from_team_data = set(team_df.team)
link_team_data = set(df2.home_team_x)
# originally I had this the other way round, which resulted in multiple matches
matched_teams_team_data = [(d,(process.extractOne(d.replace('Man.','Manchester'), teams_from_team_data))[0]) for d in link_team_data]
matched_teams_team_data

[('fc-martigues', 'FC Martigues'),
 ('ourense', 'CD Ourense'),
 ('chelsea-fc', 'Chelsea'),
 ('fidelis-andria', 'Fidelis Andria'),
 ('schalke-04', 'Schalke 04'),
 ('eintracht-trier', 'Eintracht Trier'),
 ('mulhouse', 'FC Mulhouse'),
 ('lazio', 'Lazio'),
 ('ac-mantova', 'Mantova'),
 ('atalanta', 'Atalanta'),
 ('gimnastic-tarragona', 'Gimnàstic'),
 ('sc-cambuur-leeuwarden', 'Cambuur'),
 ('perpignan', 'Perpignan'),
 ('girona-fc', 'Girona'),
 ('ejido', 'Poli Ejido'),
 ('barcelona-b', 'Barcelona B'),
 ('rimini-calcio', 'Rimini'),
 ('luton-town-fc', 'Luton Town'),
 ('omniworld', 'FC Omniworld'),
 ('az', 'AZ Alkmaar'),
 ('olympique-ales', 'Olympique Alès'),
 ('sas-epinal', 'Épinal'),
 ('bayreuth-spvgg', 'Bayreuth SpVgg'),
 ('lillestrom', 'Lille'),
 ('us-sassuolo-calcio', 'Sassuolo'),
 ('oldham-athletic-afc', 'Oldham Athletic'),
 ('fc-groningen', 'Groningen'),
 ('rbc', 'RBC Roosendaal'),
 ('fc-nurnberg', 'FC Maia'),
 ('heracles', 'Heracles'),
 ('excelsior', 'Excelsior'),
 ('valence', 'Valence')

In [18]:
def team_name_for_link_team_data(t):
    matches = [x[0] for x in matched_teams_team_data if x[1] == t]
    return (matches if len(matches)>0 else [''])[0]
team_df['matched_team'] = team_df.apply(lambda row: team_name_for_link_team_data(row['team']), axis=1)

In [19]:
# Brighton is particularly problematic! Let's see if it gets picked up
team_df[team_df.city=='Brighton']

,team,city,country,stadium,capacity,pitch,matched_team
99,Brighton Hove Alb.,Brighton,England,The American Express Community Stadium,30750,Natural,brighton-amp-hov


In [20]:
team_df[team_df['matched_team']=='crystal-palace-fc']

,team,city,country,stadium,capacity,pitch,matched_team
126,Crystal Palace,Londres,England,Selhurst Park,26309,Natural,crystal-palace-fc


In [21]:
df3 = pd.merge(df2, team_df, how="inner", left_on="home_team_x", right_on="matched_team")
df3

,link,referee,home_yellow,home_red,away_yellow,away_red,date_dt,link_parts,home_team_x,away_team_x,season_x,home_team_y,away_team_y,result,season_y,round,league,home,away,total_goals,outcome,team,city,country,stadium,capacity,pitch,matched_team
0,/match/saarbrucken/stuttgarter-kickers/1990,Hans-Jürgen Weber,0.0,0.0,3.0,0.0,1989-07-29 15:00:00,"[, match, saarbrucken, stuttgarter-kickers, 1990]",saarbrucken,stuttgarter-kickers,1990,Saarbrücken,Stuttgarter Kickers,1-0,1990,1,2_liga,1,0,1,3,Saarbrücken,Saarbrücken,Germany,Stadion Ludwigspark,35286,Natural,saarbrucken
1,/match/saarbrucken/hannover-96/1990,Eugen Strigel,0.0,0.0,3.0,0.0,1989-08-12 15:00:00,"[, match, saarbrucken, hannover-96, 1990]",saarbrucken,hannover-96,1990,Saarbrücken,Hannover 96,2-0,1990,3,2_liga,2,0,2,3,Saarbrücken,Saarbrücken,Germany,Stadion Ludwigspark,35286,Natural,saarbrucken
2,/match/saarbrucken/meppen/1990,Siegbert Rubel,0.0,0.0,3.0,0.0,1989-08-23 19:30:00,"[, match, saarbrucken, meppen, 1990]",saarbrucken,meppen,1990,Saarbrücken,Meppen,4-0,1990,5,2_liga,4,0,4,3,Saarbrücken,Saarbrücken,Germany,Stadion Ludwigspark,35286,Natural,saarbrucken
3,/match/saarbrucken/blau-weiss-1890-berlin/1990,Wilfried Heitmann,0.0,0.0,0.0,0.0,1989-09-02 15:00:00,"[, match, saarbrucken, blau-weiss-1890-berlin,...",saarbrucken,blau-weiss-1890-berlin,1990,Saarbrücken,Blau-Weiß 1890 Berlin,1-1,1990,7,2_liga,1,1,2,2,Saarbrücken,Saarbrücken,Germany,Stadion Ludwigspark,35286,Natural,saarbrucken
4,/match/saarbrucken/vfl-osnabruck/1990,Karl Gangkofer,1.0,0.0,2.0,0.0,1989-09-17 15:00:00,"[, match, saarbrucken, vfl-osnabruck, 1990]",saarbrucken,vfl-osnabruck,1990,Saarbrücken,VfL Osnabrück,7-1,1990,9,2_liga,7,1,8,3,Saarbrücken,Saarbrücken,Germany,Stadion Ludwigspark,35286,Natural,saarbrucken
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109606,/match/bourges/nancy/1994,Marc Flosi,0.0,0.0,0.0,1.0,1993-09-01 00:00:00,"[, match, bourges, nancy, 1994]",bourges,nancy,1994,Bourges 18,Nancy,2-0,1994,9,ligue_2,2,0,2,3,Bourges 18,Bourges,France,NaN,13000,NaN,bourges
109607,/match/bourges/red-star/1994,,0.0,0.0,0.0,1.0,1994-03-05 00:00:00,"[, match, bourges, red-star, 1994]",bourges,red-star,1994,Bourges 18,Red Star,0-1,1994,31,ligue_2,0,1,1,4,Bourges 18,Bourges,France,NaN,13000,NaN,bourges
109608,/match/bourges/sedan/1994,Stéphane Moulin,0.0,0.0,0.0,1.0,1993-09-11 00:00:00,"[, match, bourges, sedan, 1994]",bourges,sedan,1994,Bourges 18,Sedan,1-2,1994,10,ligue_2,1,2,3,4,Bourges 18,Bourges,France,NaN,13000,NaN,bourges
109609,/match/bourges/chamois-niortais/1994,Hervé Bayet,0.0,0.0,0.0,0.0,1993-10-27 00:00:00,"[, match, bourges, chamois-niortais, 1994]",bourges,chamois-niortais,1994,Bourges 18,Niort,1-1,1994,18,ligue_2,1,1,2,2,Bourges 18,Bourges,France,NaN,13000,NaN,bourges


# ELO

In [22]:
import pickle
elo_dict = pickle.load(open('additional-data/elo_dict.pkl', 'rb'))
elo = pd.DataFrame.from_dict(elo_dict).transpose()
elo.columns = elo.columns.str.lower()

In [23]:
elo['link_parts'] = elo.index.str.split('/')
elo['home_team'] = elo.apply(lambda row: row['link_parts'][-3], axis=1)
elo['away_team'] = elo.apply(lambda row: row['link_parts'][-2], axis=1)
elo['season'] = elo.apply(lambda row: int(str(row['link_parts'][-1])[:4]), axis=1)
elo['link_index'] = elo.apply(lambda row: str(row['link_parts'][-1])[4:], axis=1)
elo['link'] = elo.apply(lambda row: '/'.join(['','match',row['home_team'],row['away_team'],str(row['season'])]), axis=1)
elo = elo.drop(['link_parts'], axis=1)

In [24]:
elo.info()

<class 'pandas.core.frame.DataFrame'>
Index: 132111 entries, https://www.besoccer.com/match/saarbrucken/stuttgarter-kickers/19903487 to https://www.besoccer.com/match/chievo/pisa-calcio/2021103065
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   elo_home    122314 non-null  float64
 1   elo_away    122314 non-null  float64
 2   home_team   132111 non-null  object 
 3   away_team   132111 non-null  object 
 4   season      132111 non-null  int64  
 5   link_index  132111 non-null  object 
 6   link        132111 non-null  object 
dtypes: float64(2), int64(1), object(4)
memory usage: 8.1+ MB


In [25]:
df3_teams = set(df3.home_team_x)
# lower case fixes sd-Compostela which was the only non-match
elo_teams = set(elo.home_team.str.lower())
# empty set difference shows that all teams in our data so far (df3) are included in elo
df3_teams.difference(elo_teams)

set()

In [26]:
elo.home_team = elo.home_team.str.lower()

In [27]:
df4 = pd.merge(df3, elo, how="left", on="link")
# clean up - drop some of the columns created by the joins
df4.drop(['link_parts','home_team_y','away_team_y','season_y','team','matched_team','home_team','away_team','season'], axis=1, inplace=True)
df4.rename(columns={'home_team_x':'home_team','away_team_x':'away_team','season_x':'season','home':'home_goals','away':'away_goals'}, inplace=True)
df4

,link,referee,home_yellow,home_red,away_yellow,away_red,date_dt,home_team,away_team,season,result,round,league,home_goals,away_goals,total_goals,outcome,city,country,stadium,capacity,pitch,elo_home,elo_away,link_index
0,/match/saarbrucken/stuttgarter-kickers/1990,Hans-Jürgen Weber,0.0,0.0,3.0,0.0,1989-07-29 15:00:00,saarbrucken,stuttgarter-kickers,1990,1-0,1,2_liga,1,0,1,3,Saarbrücken,Germany,Stadion Ludwigspark,35286,Natural,56.0,59.0,3487
1,/match/saarbrucken/hannover-96/1990,Eugen Strigel,0.0,0.0,3.0,0.0,1989-08-12 15:00:00,saarbrucken,hannover-96,1990,2-0,3,2_liga,2,0,2,3,Saarbrücken,Germany,Stadion Ludwigspark,35286,Natural,56.0,60.0,3507
2,/match/saarbrucken/meppen/1990,Siegbert Rubel,0.0,0.0,3.0,0.0,1989-08-23 19:30:00,saarbrucken,meppen,1990,4-0,5,2_liga,4,0,4,3,Saarbrücken,Germany,Stadion Ludwigspark,35286,Natural,57.0,53.0,3533
3,/match/saarbrucken/blau-weiss-1890-berlin/1990,Wilfried Heitmann,0.0,0.0,0.0,0.0,1989-09-02 15:00:00,saarbrucken,blau-weiss-1890-berlin,1990,1-1,7,2_liga,1,1,2,2,Saarbrücken,Germany,Stadion Ludwigspark,35286,Natural,57.0,56.0,3553
4,/match/saarbrucken/vfl-osnabruck/1990,Karl Gangkofer,1.0,0.0,2.0,0.0,1989-09-17 15:00:00,saarbrucken,vfl-osnabruck,1990,7-1,9,2_liga,7,1,8,3,Saarbrücken,Germany,Stadion Ludwigspark,35286,Natural,57.0,52.0,3575
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109610,/match/bourges/nancy/1994,Marc Flosi,0.0,0.0,0.0,1.0,1993-09-01 00:00:00,bourges,nancy,1994,2-0,9,ligue_2,2,0,2,3,Bourges,France,NaN,13000,NaN,NaN,NaN,35314
109611,/match/bourges/red-star/1994,,0.0,0.0,0.0,1.0,1994-03-05 00:00:00,bourges,red-star,1994,0-1,31,ligue_2,0,1,1,4,Bourges,France,NaN,13000,NaN,NaN,NaN,35553
109612,/match/bourges/sedan/1994,Stéphane Moulin,0.0,0.0,0.0,1.0,1993-09-11 00:00:00,bourges,sedan,1994,1-2,10,ligue_2,1,2,3,4,Bourges,France,NaN,13000,NaN,NaN,NaN,35322
109613,/match/bourges/chamois-niortais/1994,Hervé Bayet,0.0,0.0,0.0,0.0,1993-10-27 00:00:00,bourges,chamois-niortais,1994,1-1,18,ligue_2,1,1,2,2,Bourges,France,NaN,13000,NaN,NaN,NaN,35411


In [28]:
fulldf = df4.copy()

# Feature engineering

## Newly-promoted teams (and newly-relegated)

Highlight which teams are new in the league this season

In [29]:
# try this for premier_league 2018 as an example
def teams(league,season):
    return set(fulldf[(fulldf['league']==league) & (fulldf['season']==season)].home_team)
pl18_teams = teams('premier_league','2018')
pl17_teams = teams('premier_league','2017')
pl18_teams.difference(pl17_teams)

{'brighton-amp-hov', 'huddersfield-town-fc', 'newcastle-united-fc'}

In [30]:
# we have to connect pairs of leagues so we can look at relegation and promotion between them

league_pairings = [
    ('ligue_1','ligue_2'),
    ('eredivisie','eerste_divisie'),
    ('premier_league','championship'),
    ('primera_division','segunda_division'),
    ('primeira_liga','segunda_liga'),
    ('serie_a','serie_b'),
    ('bundesliga','2_liga')
]

# returns the league that's connected (same country) to the one supplied, and t/f as to whether the league supplied as l is the top league
def pairing(l):
    pair = [p for p in league_pairings if l in p][0]
    return ([t for t in pair if t != l][0], l == pair[0]) 

# eg

In [31]:
def teams_new_this_season(league, season):
    previous_season = teams(league,str(int(season)-1))
    this_season = teams(league, season)
    return list(this_season.difference(previous_season))

def teams_gone_from_last_season(league, season):
    previous_season = teams(league,str(int(season)-1))
    this_season = teams(league, season)
    return list(previous_season.difference(this_season))


In [32]:
# returns a 2-tuple of (promoted teams, relegated teams) - i.e. lists of teams in this league this season who arrived by either promotion or relegation
def promoted_teams(league, season):
    previous_season = teams(league,str(int(season)-1))
    this_season = teams(league, season)
    if len(previous_season) == 0:
        return ([],[])
    new_this_season = teams_new_this_season(league, season)
    connected = pairing(league)
    if not connected[1]: # league is NOT the top league so we need to look at relegated teams too
        relegated_teams = teams_gone_from_last_season(connected[0], season)
        promoted_teams = list(set(new_this_season).difference(set(relegated_teams)))
        return (promoted_teams, relegated_teams)
    else:
        return (new_this_season, [])
    

In [33]:
promoted_teams('championship','2000')

(['fulham', 'walsall', 'manchester-city-fc'],
 ['blackburn-rovers-fc', 'charlton-athletic-fc', 'nottingham-forest-fc'])

In [34]:
leagues = list(set(fulldf.league))
seasons = list(set(fulldf.season))
df_moved_teams = pd.DataFrame(columns=leagues)
df_moved_teams['season'] = seasons
df_moved_teams.set_index('season', inplace=True)
for league in leagues:
    for season in seasons:
        df_moved_teams.at[season,league] = promoted_teams(league, season)
df_moved_teams

,eerste_divisie,serie_a,serie_b,segunda_liga,premier_league,2_liga,ligue_2,primeira_liga,primera_division,bundesliga,championship,eredivisie,segunda_division,ligue_1
season,,,,,,,,,,,,,,
2000,"([go-ahead-eagles, fc-eindhoven, ado-den-haag,...","([], [])","([savoia-1908, alzanocene, vicenza-calcio, emp...","([naval, fc-felgueiras, uniao-lamas, sporting-...","([bradford-city, sunderland-afc, watford-fc], [])","([waldhof-mannheim, chemnitzer, alemannia-aach...","([lohuans-cuiseaux, creteil-lusitanos], [socha...","([santa-clara, belenenses, gil-vicente], [])","([rayo-vallecano, sevilla, numancia, malaga], [])","([arminia-bielefeld, ulm, unterhaching], [])","([fulham, walsall, manchester-city-fc], [black...","([fc-den-bosch], [])","([elche, cordoba, getafe, levante], [tenerife,...","([troyes, saint-etienne, sedan], [])"
2003,"([ado-den-haag, helmond-sport, emmen], [fortun...","([], [])","([livorno, ascoli, catania], [hellas-verona-fc...","([portimonense, ovarense, fc-felgueiras, uniao...","([west-bromwich, manchester-city-fc, birmingha...","([lubeck, eintracht-trier, wacker-burghausen],...","([clermont-foot, stade-reims, valence, toulous...","([academica, moreirense-fc, nacional], [])","([racing, recreativo], [])","([hannover-96, arminia-bielefeld, bochum], [])","([brighton-amp-hov, reading-fc, stoke-city], [...","([fc-zwolle, rbc, excelsior], [])","([getafe, almeria, sd-compostela], [tenerife, ...","([nice, strasbourg, ajaccio], [])"
2004,"([haarlem, fc-dordrecht, sparta-rotterdam, ago...","([], [])","([treviso, como, fiorentina, us-avellino, atal...","([leixoes, estoril, desportivo-aves], [varzim,...","([portsmouth, wolverhampton], [])","([vfl-osnabruck, ssv-jahn-regensburg, unterhac...","([rouen-1899, angers-sco], [troyes, sedan])","([alverca, estrela-amadora, rio-ave], [])","([albacete, real-zaragoza, real-murcia], [])","([sc-freiburg, eintracht-frankfurt], [])","([wigan-athletic, crewe-alexandra, cardiff-cit...","([fc-volendam, ado-den-haag], [])","([cadiz, algeciras-cf, ciudad-de-murcia], [ray...","([metz, toulouse-fc], [])"
2002,"([sc-cambuur-leeuwarden, go-ahead-eagles, fc-v...","([], [])","([modena-fc, napoli, vicenza-calcio, palermo, ...","([rio-ave, chaves, moreirense-fc, nacional], [...","([fulham, bolton-wanderers-fc, blackburn-rover...","([babelsberg, karlsruher-sc, schweinfurt], [un...","([amiens-sc, istres, grenoble-foot-38], [saint...","([varzim, santa-clara, vitoria-setubal], [])","([tenerife, sevilla], [])","([fc-nurnberg], [])","([walsall, millwall-fc, rotherham-united], [br...","([fc-den-bosch], [])","([ejido, gimnastic-tarragona, xerez], [racing,...","([sochaux, montpellier-hsc, lorient], [])"
1995,"([], [])","([brescia, fiorentina, calcio-padova], [])","([], [])","([], [])","([crystal-palace-fc, nottingham-forest-fc], [])","([fortuna-dusseldorf, zwickau, fsv-frankfurt],...","([], [toulouse-fc, olympique-marsella, angers-...","([uniao-leiria, tirsense, chaves], [])","([espanyol, sd-compostela], [])","([kfc-uerdingen-05, bochum, 1860-munchen], [])","([port-vale, burnley-fc, reading-fc], [sheffie...","([fc-dordrecht, nec], [])","([cf-extremadura, salamanca, getafe, ourense],...","([stade-rennes, bastia, nice], [])"
2021,"([jong-ajax, almere-city, jong-utrecht, sc-cam...","([benevento-calcio, asd-spezia, fc-crotone], [])","([ac-monza-brianza-1912, vicenza-calcio, reggi...","([cova-piedade, estoril, feirense, vizela, aca...","([fulham, west-bromwich, leeds-united-afc], [])","([wurzburger-kickers], [fortuna-dusseldorf, pa...","([pau, dunkerque], [amiens-sc, toulouse-fc])","([farense, nacional], [])","([elche, cadiz, huesca], [])","([arminia-bielefeld, stuttgart], [])","([], [])","([], [])","([cartagena, cd-castellon, sabadell, ud-logron...","([lorient, lens], [])"
2019,"([], [])","([frosinone-calcio, empoli-fc, parma-fc], [])","([lecce, nuova-cosenza, livorno, calcio-padova...","([varzim, farense, benfica-ii, penafiel], [pac...","([fulham, wolverhampton, cardiff-city-fc], [])","([magdeburg, paderborn], [hamburger-s

In [35]:
fulldf['home_newly_promoted'] = fulldf.apply(lambda row: row.home_team in df_moved_teams.at[row.season,row.league][0], axis=1)
fulldf['home_newly_relegated'] = fulldf.apply(lambda row: row.home_team in df_moved_teams.at[row.season,row.league][1], axis=1)
fulldf['away_newly_promoted'] = fulldf.apply(lambda row: row.away_team in df_moved_teams.at[row.season,row.league][0], axis=1)
fulldf['away_newly_relegated'] = fulldf.apply(lambda row: row.away_team in df_moved_teams.at[row.season,row.league][1], axis=1)
fulldf

,link,referee,home_yellow,home_red,away_yellow,away_red,date_dt,home_team,away_team,season,result,round,league,home_goals,away_goals,total_goals,outcome,city,country,stadium,capacity,pitch,elo_home,elo_away,link_index,home_newly_promoted,home_newly_relegated,away_newly_promoted,away_newly_relegated
0,/match/saarbrucken/stuttgarter-kickers/1990,Hans-Jürgen Weber,0.0,0.0,3.0,0.0,1989-07-29 15:00:00,saarbrucken,stuttgarter-kickers,1990,1-0,1,2_liga,1,0,1,3,Saarbrücken,Germany,Stadion Ludwigspark,35286,Natural,56.0,59.0,3487,False,False,False,False
1,/match/saarbrucken/hannover-96/1990,Eugen Strigel,0.0,0.0,3.0,0.0,1989-08-12 15:00:00,saarbrucken,hannover-96,1990,2-0,3,2_liga,2,0,2,3,Saarbrücken,Germany,Stadion Ludwigspark,35286,Natural,56.0,60.0,3507,False,False,False,False
2,/match/saarbrucken/meppen/1990,Siegbert Rubel,0.0,0.0,3.0,0.0,1989-08-23 19:30:00,saarbrucken,meppen,1990,4-0,5,2_liga,4,0,4,3,Saarbrücken,Germany,Stadion Ludwigspark,35286,Natural,57.0,53.0,3533,False,False,False,False
3,/match/saarbrucken/blau-weiss-1890-berlin/1990,Wilfried Heitmann,0.0,0.0,0.0,0.0,1989-09-02 15:00:00,saarbrucken,blau-weiss-1890-berlin,1990,1-1,7,2_liga,1,1,2,2,Saarbrücken,Germany,Stadion Ludwigspark,35286,Natural,57.0,56.0,3553,False,False,False,False
4,/match/saarbrucken/vfl-osnabruck/1990,Karl Gangkofer,1.0,0.0,2.0,0.0,1989-09-17 15:00:00,saarbrucken,vfl-osnabruck,1990,7-1,9,2_liga,7,1,8,3,Saarbrücken,Germany,Stadion Ludwigspark,35286,Natural,57.0,52.0,3575,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109610,/match/bourges/nancy/1994,Marc Flosi,0.0,0.0,0.0,1.0,1993-09-01 00:00:00,bourges,nancy,1994,2-0,9,ligue_2,2,0,2,3,Bourges,France,NaN,13000,NaN,NaN,NaN,35314,False,False,False,False
109611,/match/bourges/red-star/1994,,0.0,0.0,0.0,1.0,1994-03-05 00:00:00,bourges,red-star,1994,0-1,31,ligue_2,0,1,1,4,Bourges,France,NaN,13000,NaN,NaN,NaN,35553,False,False,False,False
109612,/match/bourges/sedan/1994,Stéphane Moulin,0.0,0.0,0.0,1.0,1993-09-11 00:00:00,bourges,sedan,1994,1-2,10,ligue_2,1,2,3,4,Bourges,France,NaN,13000,NaN,NaN,NaN,35322,False,False,False,False
109613,/match/bourges/chamois-niortais/1994,Hervé Bayet,0.0,0.0,0.0,0.0,1993-10-27 00:00:00,bourges,chamois-niortais,1994,1-1,18,ligue_2,1,1,2,2,Bourges,France,NaN,13000,NaN,NaN,NaN,35411,False,False,False,False


In [36]:
# drop the items missing elo
print(fulldf.size)
fulldf.dropna(subset=['elo_home','elo_away'],inplace=True)
print(fulldf.size)

3178835
2970093


In [37]:
fulldf.sort_values('date_dt',inplace=True)

In [38]:
fulldf[['home_goals_cum','away_goals_cum']] = fulldf.groupby(by=['season','home_team']).cumsum()[['home_goals','away_goals']]
# example
# fulldf[(fulldf['season']=='2018') & (fulldf['home_team']=='brighton-amp-hov')]

# Winning / losing streaks

In [39]:
df5 = fulldf.copy().set_index(['season','league']).sort_values('date_dt')

In [40]:
# actually it looks like I don't use this at all
# so I'll comment it out for now - remove later 
# df5['home_win'] = df5.apply(lambda row: 1 if row['home_goals'] > row['away_goals'] else 0, axis=1)
# df5['away_win'] = df5.apply(lambda row: 1 if row['home_goals'] < row['away_goals'] else 0, axis=1)
# df5['draw'] = df5.apply(lambda row: 1 if row['home_goals'] == row['away_goals'] else 0, axis=1)

In [41]:
teams = list(df5['home_team'].unique())

In [44]:
# This construction of `team_res` gives decent raw data (win/lose streaks regardless of whether each game was home or away). It requires a column for every team in the dataset.
df6 = df5.copy().reset_index()
teams = list(df6['home_team'].unique())
# Create the new columns, one per team and a row per match, then concat them onto the main data
z = np.zeros(shape=(len(df6),len(teams)))
team_cols = pd.DataFrame(z, columns = teams)
team_results_df = pd.concat([df6, team_cols],axis=1)
# fill in each team's column with 1 for a win and -1 for a defeat against that match
# we aren't gonna count draws for now
# so the majority of the new columns will just contain a zero, and only if there's a non-draw result will there be two cols containing a 1 / -1
team_results_df.reset_index(inplace=True)
for i in team_results_df.index:
    team_results_df.loc[i, team_results_df.loc[i, 'home_team']] = 1 if team_results_df.loc[i, 'home_goals'] > team_results_df.loc[i, 'away_goals'] else -1 if team_results_df.loc[i, 'home_goals'] < team_results_df.loc[i, 'away_goals'] else 0
    team_results_df.loc[i, team_results_df.loc[i, 'away_team']] = 1 if team_results_df.loc[i, 'home_goals'] < team_results_df.loc[i, 'away_goals'] else -1 if team_results_df.loc[i, 'home_goals'] > team_results_df.loc[i, 'away_goals'] else 0

In [45]:
team_results_df.reset_index(inplace=True)
team_results_df.set_index(['season'],inplace=True)
team_results_df.sort_values('date_dt',inplace=True)
team_results_df.head()

,level_0,index,league,link,referee,home_yellow,home_red,away_yellow,away_red,date_dt,home_team,away_team,result,round,home_goals,away_goals,total_goals,outcome,city,country,stadium,capacity,pitch,elo_home,elo_away,link_index,home_newly_promoted,home_newly_relegated,away_newly_promoted,away_newly_relegated,home_goals_cum,away_goals_cum,home_win,away_win,draw,olympique-lyonnais,racing-paris,sochaux,lillestrom,nantes,sporting-toulon,toulouse-fc,montpellier-hsc,metz,kfc-uerdingen-05,eintracht-frankfurt,fortuna-dusseldorf,bayern-munchen,bayer-leverkusen,alemannia-aachen,saarbrucken,sc-freiburg,fortuna-koln,hertha-bsc,hessen-kassel,rot-weiss-essen,wattenscheid-09,vfl-osnabruck,stuttgart,hamburger-sv,werder-bremen,homburg,waldhof-mannheim,karlsruher-sc,fc-nurnberg,unterhaching,preuben-munster,hannover-96,darmstadt-98,blau-weiss-1890-berlin,bayreuth-spvgg,meppen,msv-duisburg,kaiserslautern,mvv,bochum,fc-twente-1965,ado-den-haag,sparta-rotterdam,vitesse,rkc-waalwijk,fc-utrecht,fc-volendam,haarlem,fc-den-bosch,willem-ii,borussia-dortmund,roda-jc,feyenoord,ajax,schalke-04,hull-city,watford-fc,west-bromwich,middlesbrough-fc,newcastle-united-fc,blackburn-rovers-fc,stoke-city,bradford-city,swindon-town,ipswich-town-fc,brighton-amp-hov,charlton-athletic-fc,southampton-fc,queens-park-rangers-fc,nottingham-forest-fc,feirense,portimonense,sheffield-wednesday-fc,sporting-braga,tottenham-hotspur-fc,wimbledon-fc,liverpool,nec,coventry-city,chaves,fc-groningen,tirsense,maritimo,luton-town-fc,arsenal,millwall-fc,sunderland-afc,afc-bournemouth,chelsea-fc,everton-fc,crystal-palace-fc,oldham-athletic-afc,psv,west-ham-united,aston-villa-fc,manchester-city-fc,fortuna-sittard,norwich-city-fc,derby-county-fc,leeds-united-afc,vitoria-guimaraes,penafiel,portsmouth,port-vale,boavista,vitoria-setubal,oxford-united,sheffield-united,barnsley-fc,wolverhampton,belenenses,uniao-madeira,lazio,udinese,atalanta,genoa,ascoli,ac-cesena,juventus-fc,nacional,estrela-amadora,internazionale,cd-malaga,plymouth-argyle,valladolid,cadiz,lecce,deportivo,beira-mar,napoli,roma,bologna,hellas-verona-fc,sampdoria,figueres,osasuna,us-cremonese,racing,sabadell,levante,fiorentina,real-zaragoza,real-burgos-cf,sevilla,real-oviedo,milan,espanyol,cd-castellon,barcelona,rm-castilla,rayo-vallecano,benfica,celta,eibar,athletic-bilbao-b,recreativo,mallorca,salamanca,logrones-cd,elche,sestao-sport-club,real-murcia,real-madrid,valencia-cf,tenerife,xerez,bordeaux,monaco,as-cannes,saint-etienne,olympique-marsella,caen,nancy,auxerre,nice,schweinfurt,stade-brestois-29,havelse,stade-rennes,vfb-oldenburg,farense,famalicao,heerenveen,bristol-city-fc,salgueiros,gil-vicente,svv,bristol-rovers,notts-county,orihuela,real-aviles-ind,parma-fc,cagliari,torino-fc,ue-lleida,albacete,pisa-calcio,lens,lokomotive-leipzig,hansa-rostock,sg-dynamo-dresden,1860-munchen,hallescher-fc,vvv,grimsby-town,southend-united,pacos-ferreira,graafschap,tranmere-rovers,torreense,estoril,fc-dordrecht,fc-carl-zeiss-jena,cambridge-united,merida-cp,barcelona-b,chemnitzer,wolfsburg,fc-remscheid,wuppertaler-sv,strasbourg,go-ahead-eagles,brentford,valenciennes-afc,peterborough-u,birmingham-city-fc,sc-cambuur-leeuwarden,espinho,atletico-marbella,badajoz,pescara-calcio,brescia,ancona,lugo,villarreal,fc-martigues,tennis-borussia,angers-sco,nac-breda,oxford-city,bolton-wanderers-fc,piacenza,leganes,hercules,cd-toledo,bastia,uniao-leiria,zwickau,fsv-frankfurt,como,getafe,palermo,salernitana-calcio-1919,chievo,perugia,cf-extremadura,vicenza-calcio,calcio-padova,acireale,venezia,fidelis-andria,ourense,berrichonne-chateauroux,stade-lavallois,fc-gueugnon,guingamp,lorient,mulhouse,olympique-ales,red-star,chamois-niortais,dunkerque,sas-epinal,amiens-sc,perpignan,charleville,lohuans-cuiseaux,valence,arminia-bielefeld,campomaiorense,fc-felgueiras,lubeck,reggina,us-avellino,almeria,alaves,pistoiese,gutersloh,veendam,heracles,helmond-sport,emmen,az,fc-eindhoven,rio-ave,empoli-fc,castel-sangro,energie-cottbus,greuther-furth,fc-zwolle,stormvogels-telstar

In [46]:
team_results_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 102417 entries, 1990 to 2021
Columns: 526 entries, level_0 to cultural-deportiva-leonesa
dtypes: bool(4), datetime64[ns](1), float64(497), int64(13), object(11)
memory usage: 409.1+ MB


In [47]:
# so this is almost quite nice ... giving a negative count for successive defeats and a positive count for successive wins
# the only flaw really is that draws just give zeros - we'll live with that for now, taking a draw as a neutral thing rather than a streak (which isn't really right)
# this code here is just a proof-of-concept for what's used in the cell below as the real function
season = '2018'
team = 'brighton-amp-hov'
filtered = team_results_df.loc[season].query('home_team == @team or away_team == @team')
filtered = filtered[['link','result',team,'round','date_dt']]
filtered['streak'] = filtered[team].groupby((filtered[team]!=filtered[team].shift()).cumsum()).cumsum()
filtered

,link,result,brighton-amp-hov,round,date_dt,streak
season,,,,,,
2018,/match/brighton-amp-hov/manchester-city-fc/2018,0-2,-1.0,1,2017-08-12 18:30:00,-1.0
2018,/match/watford-fc/brighton-amp-hov/2018,0-0,0.0,3,2017-08-26 16:00:00,0.0
2018,/match/brighton-amp-hov/west-bromwich/2018,3-1,1.0,4,2017-09-09 16:00:00,1.0
2018,/match/afc-bournemouth/brighton-amp-hov/2018,2-1,-1.0,5,2017-09-15 21:00:00,-1.0
2018,/match/brighton-amp-hov/newcastle-united-fc/2018,1-0,1.0,6,2017-09-24 17:00:00,1.0
2018,/match/arsenal/brighton-amp-hov/2018,2-0,-1.0,7,2017-10-01 13:00:00,-1.0
2018,/match/brighton-amp-hov/everton-fc/2018,1-1,0.0,8,2017-10-15 14:30:00,0.0
2018,/match/west-ham-united/brighton-amp-hov/2018,0-3,1.0,9,2017-10-20 21:00:00,1.0
2018,/match/brighton-amp-hov/southampton-fc/2018,1-1,0.0,10,2017-10-29 14:30:00,0.0


In [48]:
# create new cols containing 
def count_streaks(season, team):
    filtered = team_results_df.loc[season].query('home_team == @team or away_team == @team')
    filtered = filtered[['link','result',team,'round','date_dt']].sort_values('date_dt')
    # calculate streaks
    filtered['start_of_streak'] = filtered[team].ne(filtered[team].shift())
    filtered['streak_id'] = filtered['start_of_streak'].cumsum()
    filtered['streak_result'] = filtered[team]
    filtered['streak_counter'] = filtered.groupby('streak_id').cumcount()
    filtered = filtered[['link','streak_counter','streak_result',team]]
    filtered.rename({'streak_counter': team+'_streak','streak_result':team+'_streak_result'},axis=1,inplace=True)
    return filtered


In [49]:
# test it
count_streaks('2018','brighton-amp-hov')

,link,brighton-amp-hov_streak,brighton-amp-hov_streak_result,brighton-amp-hov
season,,,,
2018,/match/brighton-amp-hov/manchester-city-fc/2018,0,-1.0,-1.0
2018,/match/watford-fc/brighton-amp-hov/2018,0,0.0,0.0
2018,/match/brighton-amp-hov/west-bromwich/2018,0,1.0,1.0
2018,/match/afc-bournemouth/brighton-amp-hov/2018,0,-1.0,-1.0
2018,/match/brighton-amp-hov/newcastle-united-fc/2018,0,1.0,1.0
2018,/match/arsenal/brighton-amp-hov/2018,0,-1.0,-1.0
2018,/match/brighton-amp-hov/everton-fc/2018,0,0.0,0.0
2018,/match/west-ham-united/brighton-amp-hov/2018,0,1.0,1.0
2018,/match/brighton-amp-hov/southampton-fc/2018,0,0.0,0.0


In [50]:
# do the count_streaks fn for each team in each season
# and collect the dfs in a list
cumul_dfs = []
for season in seasons:
    for team in teams:
        cumul_dfs.append(count_streaks(str(season), team))


In [51]:
# drop the empty ones (dunno why there are so many?? perhaps this is where there aren't any streaks so everything zero, but I suspect it's something more sinister???)
print(len(cumul_dfs))
streak_dfs = [df for df in cumul_dfs if not df.empty]
print(len(streak_dfs))

15136
6522


In [52]:
# concat all those dfs together so we've got all the teams' streaks 
# in this combined df we'll get all the different columns back, most of which will be full of NaN, but one set of 3 cols in each row will have content
all_streaks_df = pd.concat(streak_dfs)
all_streaks_df.reset_index(inplace=True)

/var/folders/3m/mch0swq10qd743jg357mbd7h0000gp/T/ipykernel_58020/2735847363.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_streaks_df.reset_index(inplace=True)


In [53]:
# check what it looks like
all_streaks_df.loc[0]

season                                                                        2000
link                                /match/olympique-lyonnais/montpellier-hsc/2000
olympique-lyonnais_streak                                                      0.0
olympique-lyonnais_streak_result                                              -1.0
olympique-lyonnais                                                            -1.0
                                                         ...                      
as-gubbio-1910_streak_result                                                   NaN
as-gubbio-1910                                                                 NaN
granada-74_streak                                                              NaN
granada-74_streak_result                                                       NaN
granada-74                                                                     NaN
Name: 0, Length: 1421, dtype: object

In [54]:
# return a pair of values, representing the home streak = length * sign (positive for win, negative for loss) , away streak
# each row will have only one team, and so either home or away will be zero, the other will be real data (possibly zero if either no streak or a streak of draws)
def home_or_away(row):
    # which team cols are non-NaN?
    first_non_null_col = row[2:].first_valid_index()
    team_name = first_non_null_col.split('_')[0]
    link_parts = row['link'].split('/')
    # print(link_parts, team_name)
    home_away = 'home' if team_name==link_parts[2] else 'away'
    # we'll return home_streak * home_streak_result, away_streak * away_streak_result
    if home_away == 'home':
        return row[team_name + '_streak'] * row[team_name + '_streak_result'], 0
    else:
        return 0, row[team_name + '_streak'] * row[team_name + '_streak_result']

In [55]:
# arbitrary check
locno = 24
print(all_streaks_df.loc[locno])
print(home_or_away(all_streaks_df.loc[locno]))

season                                                              2000
link                                /match/nancy/olympique-lyonnais/2000
olympique-lyonnais_streak                                            0.0
olympique-lyonnais_streak_result                                    -1.0
olympique-lyonnais                                                  -1.0
                                                    ...                 
as-gubbio-1910_streak_result                                         NaN
as-gubbio-1910                                                       NaN
granada-74_streak                                                    NaN
granada-74_streak_result                                             NaN
granada-74                                                           NaN
Name: 24, Length: 1421, dtype: object
(0, -0.0)


In [56]:
# use this function to add the home_stream and away_streak cols to the df
all_streaks_df['home_streak'], all_streaks_df['away_streak'] = zip(*all_streaks_df.apply(home_or_away, axis=1))

In [57]:
# and then just keep the key columns, drop the teams now that they've been copied into the home_streak and away_streak cols - we're done with them
just_streaks_df = all_streaks_df[['link','home_streak','away_streak']].copy()

In [58]:
# combine the pairs together - for the same match (=link) we have the home_streak in one row and away_streak in another
merged_streaks_df = just_streaks_df.groupby('link').sum()
# check this
merged_streaks_df.head()

,home_streak,away_streak
link,,
/match/1860-munchen/1-fc-union-berlin/2010,0.0,0.0
/match/1860-munchen/1-fc-union-berlin/2011,1.0,0.0
/match/1860-munchen/1-fc-union-berlin/2012,0.0,0.0
/match/1860-munchen/1-fc-union-berlin/2013,0.0,0.0
/match/1860-munchen/1-fc-union-berlin/2014,3.0,0.0


In [59]:
# here's an example in the head above for us to check
# if your head comes up differently, just change the match to see what's happening
# but you should end up with two rows matching the link, one with a home_streak and the other with an away_streak
just_streaks_df[just_streaks_df['link']=='/match/1-fc-union-berlin/1860-munchen/2012']

,link,home_streak,away_streak


In [60]:
# merge that data onto the main df
df7 = df6.merge(merged_streaks_df, on='link')
df7.head()

,season,league,link,referee,home_yellow,home_red,away_yellow,away_red,date_dt,home_team,away_team,result,round,home_goals,away_goals,total_goals,outcome,city,country,stadium,capacity,pitch,elo_home,elo_away,link_index,home_newly_promoted,home_newly_relegated,away_newly_promoted,away_newly_relegated,home_goals_cum,away_goals_cum,home_win,away_win,draw,home_streak,away_streak
0,1990,ligue_1,/match/olympique-lyonnais/olympique-marsella/1990,Michel Vautrot,2.0,0.0,2.0,0.0,1989-07-21,olympique-lyonnais,olympique-marsella,1-4,1,1,4,5,4,Lyon,France,Parc Olympique Lyonnais,59168,Natural,72.0,81.0,3107,False,False,False,False,1,4,0,1,0,0.0,0.0
1,1990,ligue_1,/match/racing-paris/monaco/1990,Jean-Marie Lartigot,1.0,0.0,1.0,0.0,1989-07-22,racing-paris,monaco,0-0,1,0,0,0,1,París,France,NaN,14000,Grass,72.0,82.0,3116,False,False,False,False,0,0,0,0,1,0.0,0.0
2,1990,ligue_1,/match/sochaux/stade-brestois-29/1990,Robert Wurtz,1.0,0.0,2.0,0.0,1989-07-22,sochaux,stade-brestois-29,1-0,1,1,0,1,3,Sochaux,France,Stade Auguste-Bonal,20005,Natural,78.0,73.0,3113,False,False,False,False,1,0,1,0,0,0.0,0.0
3,1990,ligue_1,/match/lillestrom/caen/1990,Jean-Louis Rideau,3.0,0.0,2.0,0.0,1989-07-22,lillestrom,caen,1-0,1,1,0,1,3,Lille,France,Stade Pierre-Mauroy,49834,Natural,76.0,72.0,3112,False,False,False,False,1,0,1,0,0,0.0,0.0
4,1990,ligue_1,/match/nantes/auxerre/1990,JoÃ«l Quiniou,1.0,0.0,2.0,0.0,1989-07-22,nantes,auxerre,2-1,1,2,1,3,3,Nantes,France,Stade de la Beaujoire - Louis Fonteneau,38285,Natural,79.0,79.0,3111,False,False,False,False,2,1,1,0,0,0.0,0.0


In [61]:
# and check out the data for one season and one team, which is where we can check it
season = '2018'
team = 'brighton-amp-hov'
df7[(df7['season']==season) & ((df7['home_team']==team) | (df7['away_team']==team))].sort_values('date_dt')[['link','home_team','away_team','result','home_streak','away_streak']]

,link,home_team,away_team,result,home_streak,away_streak
89991,/match/brighton-amp-hov/manchester-city-fc/2018,brighton-amp-hov,manchester-city-fc,0-2,0.0,0.0
90115,/match/watford-fc/brighton-amp-hov/2018,watford-fc,brighton-amp-hov,0-0,0.0,0.0
90232,/match/brighton-amp-hov/west-bromwich/2018,brighton-amp-hov,west-bromwich,3-1,0.0,0.0
90303,/match/afc-bournemouth/brighton-amp-hov/2018,afc-bournemouth,brighton-amp-hov,2-1,0.0,0.0
90496,/match/brighton-amp-hov/newcastle-united-fc/2018,brighton-amp-hov,newcastle-united-fc,1-0,0.0,0.0
90573,/match/arsenal/brighton-amp-hov/2018,arsenal,brighton-amp-hov,2-0,1.0,0.0
90689,/match/brighton-amp-hov/everton-fc/2018,brighton-amp-hov,everton-fc,1-1,0.0,0.0
90724,/match/west-ham-united/brighton-amp-hov/2018,west-ham-united,brighton-amp-hov,0-3,0.0,0.0
90880,/match/brighton-amp-hov/southampton-fc/2018,brighton-amp-hov,southampton-fc,1-1,0.0,0.0
90944,/match/swansea-city-afc/brighton-amp-hov/2018,swansea-city-afc,brighton-amp-hov,0-1,-2.0,0.0
